In [1]:
import json
import altair as alt
from altair import expr, datum
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests

In [608]:
LOCAL = False

if LOCAL:
    local_suffix = "_local"
else:
    local_suffix = ""

In [609]:
%%capture pwd
!pwd

In [610]:
# uid = "2021-05-05-which-firms-and-industries-have-been-most-affected-by-covid-update"  # article unique ID
uid = pwd.stdout.split("/")[-1].split("\r")[0]
eco_git_home = (
    "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/"
)
eco_git_path = eco_git_home + "articles/" + uid + "/data/"
vega_embed = requests.get(eco_git_home + "guidelines/html/vega-embed.html").text
colors = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-colors.json").content
)
category_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-category-color.json").content
)
hue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-single-hue-color.json").content
)
mhue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-multi-hue-color.json").content
)
div_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-diverging-color.json").content
)
config = json.loads(
    requests.get(eco_git_home + "guidelines/charts/eco-global-config.json").content
)
height = config["height"]
width = config["width"]
height, width

(300, 500)

# Fig 1

In [611]:
df = (
    pd.read_excel(
        "raw/Economic Observatory Graphs - Mar & Apr.xlsx",
        sheet_name="WFH graphs",
        skiprows=1,
        nrows=3,
    )
    .dropna(how="all", axis=1)
    .dropna(how="all", axis=0)
)
df.columns = ["response", "percent"]
# df["percent"] *= 100

In [612]:
f = "fig1_WFH"
f1 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f1.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f1 = df
df.head()

,response,percent
0,Working from home,0.468
1,Working on business premises,0.401
2,Not working,0.131


In [613]:
bars = (
    alt.Chart(f1)
    .mark_bar(height=40, opacity=0.9)
    .encode(
        color=alt.Color(
            "response:N",
            legend=alt.Legend(title="", padding=1),
            scale=alt.Scale(
                domain=df["response"].unique(),
                range=[colors["eco-turquiose"], colors["eco-blue"], colors["eco-red"]],
            ),
        ),
        x=alt.X(
            "sum(percent):Q",
            sort=[],
            axis=alt.Axis(
                grid=False,
                title="% of respondents",
                format=".0%",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                titleAnchor="end",
                titleY=20,
                titleFontWeight="normal",
                titleFontSize=10,
            ),
            scale=alt.Scale(domain=[0, 1]),
        ),
        order="response:N",
    )
)
text = bars.mark_text(dx=-5, dy=1, align="right").encode(
    text=alt.Text("percent:Q", format=".0%"),
    color=alt.condition(
        datum.SAmmount < 8,
        alt.ColorValue(None),
        alt.ColorValue("white"),
    ),
    x=alt.X(
        "sum(percent):Q",
        stack="zero",
    ),
)
layer1 = ((bars + text).properties(height=40, width=300)).configure_legend(
    titleColor=colors["eco-gray"], labelColor=colors["eco-gray"]
)
layer1.save("visualisation/" + f + ".json")
layer1

alt.LayerChart(...)

# Fig 2

In [614]:
df = (
    pd.read_excel(
        "raw/Economic Observatory Graphs - Mar & Apr.xlsx",
        sheet_name="WFH graphs",
        skiprows=7,
        nrows=6,
    )
    .dropna(how="all", axis=1)
    .dropna(how="all", axis=0)
)
df.columns = ["response", "percent"]

In [615]:
f = "fig2_future_work"
f2 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f2.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f2 = df
df.head()

,response,percent
0,Rarely or never,0.225
1,1 day per week,0.106
2,2 days per week,0.190
3,3 days per week,0.196
4,4 days per week,0.086


In [616]:
bars = (
    alt.Chart(f2)
    .mark_bar(height=20, opacity=0.9)
    .encode(
        color=alt.Color(
            "response:N",
            legend=None,
            scale=alt.Scale(
                domain=df["response"].unique(),
                range=mhue_color + [colors["eco-dark-blue"]],
            ),
        ),
        y=alt.Y(
            "response:N",
            sort=[],
            axis=alt.Axis(
                grid=False,
                title="",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
            ),
        ),
        x=alt.X(
            "sum(percent):Q",
            sort=[],
            axis=alt.Axis(
                grid=False,
                title="of respondents",
                format=".0%",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                titleAnchor="end",
                titleY=-17,
                titleFontWeight="normal",
                titleFontSize=10,
            ),
            scale=alt.Scale(domain=[0, 0.25]),
        ),
        order="response:N",
    )
)
text = bars.mark_text(dx=-5, dy=1, align="right").encode(
    text=alt.Text("percent:Q", format=".0%"),
    color=alt.condition(
        datum.percent < 0.08,
        alt.ColorValue(None),
        alt.ColorValue("white"),
    ),
    x=alt.X(
        "sum(percent):Q",
        stack="zero",
    ),
)
layer1 = (
    ((bars + text).properties(height=alt.Step(25), width=300))
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1

alt.LayerChart(...)

# Fig 3

In [617]:
df = (
    pd.read_excel(
        "raw/Economic Observatory Graphs - Mar & Apr.xlsx",
        sheet_name="WFH graphs",
        skiprows=16,
        nrows=7,
    )
    .dropna(how="all", axis=1)
    .dropna(how="all", axis=0)
)
df.columns = ["response", "percent"]

In [618]:
f = "fig3_future_23"
f3 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f3.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f3 = df
df.head()

,response,percent
0,Incredibly\npositive\n(> 35% raise),0.015
1,Strongly\npositive\n(15 - 35% raise),0.078
2,Positive\n(< 15% raise),0.607
3,Neutral,0.225
4,Negative\n(< 15% cut),0.064


In [619]:
bars = (
    alt.Chart(f3)
    .mark_bar(height=20, opacity=0.9)
    .encode(
        color=alt.Color(
            "response:N",
            legend=None,
            scale=alt.Scale(
                domain=df["response"].unique(),
                range=[colors["eco-dark-blue"], colors["eco-mid-blue"]]
                + div_color[::-1][1:]
                + [colors["eco-purple"]],
            ),
        ),
        y=alt.Y(
            "response:N",
            sort=[],
            axis=alt.Axis(
                grid=False,
                title="",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
            ),
            scale=alt.Scale(domain=df.response.values),
        ),
        x=alt.X(
            "sum(percent):Q",
            sort=[],
            axis=alt.Axis(
                grid=False,
                title="% of respondents",
                format=".0%",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                titleAnchor="end",
                titleY=-17,
                titleFontWeight="normal",
                titleFontSize=10,
            ),
            scale=alt.Scale(domain=[0, 0.6]),
        ),
        order="response:N",
    )
)
text = bars.mark_text(dx=-5, dy=1, align="right").encode(
    text=alt.Text("percent:Q", format=".0%"),
    color=alt.condition(
        datum.percent < 0.24,
        alt.ColorValue(None),
        alt.ColorValue("white"),
    ),
    x=alt.X(
        "sum(percent):Q",
        stack="zero",
    ),
)
text2 = bars.mark_text(dx=5, dy=1, align="left").encode(
    text=alt.Text("percent:Q", format=".0%"),
    color=alt.condition(
        datum.percent > 0.24,
        alt.ColorValue(None),
        alt.ColorValue(colors["eco-gray"]),
    ),
    x=alt.X(
        "sum(percent):Q",
        stack="zero",
    ),
)
line = (
    alt.Chart(pd.DataFrame([{"x": 0, "y": "Neutral"}, {"x": 0.4, "y": "Neutral"}]))
    .mark_line(yOffset=-10, color=colors["eco-blue"])
    .encode(x=alt.X("x:Q"), y=alt.Y("y:N"))
)
avg = (
    alt.Chart(pd.DataFrame([{"x": 0.40, "y": "Neutral", "t": "Weighted average:"}]))
    .mark_text(yOffset=-6, xOffset=5, align="left", size=10, color=colors["eco-blue"])
    .encode(x=alt.X("x:Q"), y=alt.Y("y:N"), text="t:N")
)
avg2 = (
    alt.Chart(pd.DataFrame([{"x": 0.40, "y": "Neutral", "t": "6.31% pay rise"}]))
    .mark_text(yOffset=7, xOffset=5, align="left", size=10, color=colors["eco-blue"])
    .encode(x=alt.X("x:Q"), y=alt.Y("y:N"), text="t:N")
)
layer1 = (
    (
        (bars + text + text2 + line + avg + avg2).properties(
            height=alt.Step(25), width=300
        )
    )
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1

alt.LayerChart(...)

# Fig 4

In [620]:
df = (
    pd.read_excel(
        "raw/Economic Observatory Graphs - Mar & Apr.xlsx",
        sheet_name="Commuting graphs",
        skiprows=1,
        nrows=7,
    )
    .dropna(how="all", axis=1)
    .dropna(how="all", axis=0)
)
df.columns = ["response", "percent"]

In [621]:
f = "fig4_days_commuting"
f4 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f4.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f4 = df
df.head()

,response,percent
0,0 days,0.095
1,1 day,0.013
2,2 days,0.037
3,3 days,0.100
4,4 days,0.147


In [622]:
bars = (
    alt.Chart(f4)
    .mark_bar(height=20, opacity=0.9)
    .encode(
        color=alt.Color(
            "response:N",
            legend=None,
            scale=alt.Scale(
                domain=df["response"].unique(),
                range=mhue_color + [colors["eco-dark-blue"]],
            ),
        ),
        y=alt.Y(
            "response:N",
            sort=[],
            axis=alt.Axis(
                grid=False,
                title="",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
            ),
        ),
        x=alt.X(
            "sum(percent):Q",
            sort=[],
            axis=alt.Axis(
                grid=False,
                title="% of respondents",
                format=".0%",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                titleAnchor="end",
                titleY=20,
                titleFontWeight="normal",
                titleFontSize=10,
            ),
            scale=alt.Scale(domain=[0, 0.6]),
        ),
        order="response:N",
    )
)
text = bars.mark_text(dx=-5, dy=1, align="right").encode(
    text=alt.Text("percent:Q", format=".0%"),
    color=alt.condition(
        datum.percent < 0.08,
        alt.ColorValue(None),
        alt.ColorValue("white"),
    ),
    x=alt.X(
        "sum(percent):Q",
        stack="zero",
    ),
)

text2 = bars.mark_text(dx=5, dy=1, align="left").encode(
    text=alt.Text("percent:Q", format=".0%"),
    color=alt.condition(
        datum.percent > 0.08,
        alt.ColorValue(None),
        alt.ColorValue(colors["eco-gray"]),
    ),
    x=alt.X(
        "sum(percent):Q",
        stack="zero",
    ),
)
layer1 = (
    ((bars + text + text2).properties(height=alt.Step(25), width=300))
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1

alt.LayerChart(...)

# Fig 5

In [623]:
df = (
    pd.read_excel(
        "raw/Economic Observatory Graphs - Mar & Apr.xlsx",
        sheet_name="Commuting graphs",
        skiprows=10,
        nrows=5,
    )
    .dropna(how="all", axis=1)
    .dropna(how="all", axis=0)
)
df.columns = ["response", "percent"]

In [624]:
f = "fig5_time_commuting"
f5 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f5.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f5 = df
df.head()

,response,percent
0,15 minutes or less,0.282
1,16-30 minutes,0.322
2,31-45 minutes,0.192
3,46-60 minutes,0.128
4,More than 60 minutes,0.076


In [625]:
bars = (
    alt.Chart(f5)
    .mark_bar(height=20, opacity=0.9)
    .encode(
        color=alt.Color(
            "response:N",
            legend=None,
            scale=alt.Scale(
                domain=df["response"].unique(),
                range=[
                    colors["eco-green"],
                    colors["eco-yellow"],
                    colors["eco-orange"],
                    colors["eco-pink"],
                    colors["eco-red"],
                ],
            ),
        ),
        y=alt.Y(
            "response:N",
            sort=[],
            axis=alt.Axis(
                grid=False,
                title="",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
            ),
        ),
        x=alt.X(
            "sum(percent):Q",
            sort=[],
            axis=alt.Axis(
                grid=False,
                title="% of respondents",
                format=".0%",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                titleAnchor="end",
                titleY=-17,
                titleFontWeight="normal",
                titleFontSize=10,
            ),
            scale=alt.Scale(domain=[0, 0.35]),
        ),
        order="response:N",
    )
)
text = bars.mark_text(dx=-5, dy=1, align="right").encode(
    text=alt.Text("percent:Q", format=".0%"),
    color=alt.condition(
        datum.percent < 0.02,
        alt.ColorValue(None),
        alt.ColorValue("white"),
    ),
    x=alt.X(
        "sum(percent):Q",
        stack="zero",
    ),
)
line = (
    alt.Chart(
        pd.DataFrame(
            [{"x": 0, "y": "31-45 minutes"}, {"x": 0.35, "y": "31-45 minutes"}]
        )
    )
    .mark_line(yOffset=-16, color=colors["eco-blue"])
    .encode(x=alt.X("x:Q"), y=alt.Y("y:N"))
)
avg = (
    alt.Chart(
        pd.DataFrame([{"x": 0.34, "y": "31-45 minutes", "t": "Weighted average:"}])
    )
    .mark_text(yOffset=-6, xOffset=5, align="right", size=10, color=colors["eco-blue"])
    .encode(x=alt.X("x:Q"), y=alt.Y("y:N"), text="t:N")
)
avg2 = (
    alt.Chart(pd.DataFrame([{"x": 0.335, "y": "31-45 minutes", "t": "29 minutes"}]))
    .mark_text(yOffset=7, xOffset=5, align="right", size=10, color=colors["eco-blue"])
    .encode(x=alt.X("x:Q"), y=alt.Y("y:N"), text="t:N")
)
layer1 = (
    ((bars + text + line + avg + avg2).properties(height=alt.Step(25), width=300))
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1

alt.LayerChart(...)

# Fig 6

In [626]:
df = (
    pd.read_excel(
        "raw/Economic Observatory Graphs - Mar & Apr.xlsx",
        sheet_name="Commuting graphs",
        skiprows=19,
        nrows=8,
    )
    .dropna(how="all", axis=1)
    .dropna(how="all", axis=0)
)
df.columns = ["response", "percent"]

In [627]:
f = "fig6_daily_cost"
f6 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f6.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f6 = df
df.head()

,response,percent
0,0-2,0.325
1,3-5,0.364
2,6-8,0.122
3,9-11,0.087
4,12-14,0.025


In [628]:
bars = (
    alt.Chart(f6)
    .mark_bar(width=40, opacity=0.9, color=colors["eco-turquiose"])
    .encode(
        x=alt.X(
            "response:N",
            sort=[],
            axis=alt.Axis(
                grid=False,
                title="Full daily cost (£)",
                labelAngle=0,
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                titleY=20,
                titleFontWeight="normal",
                titleFontSize=10,
            ),
        ),
        y=alt.Y(
            "sum(percent):Q",
            sort=[],
            axis=alt.Axis(
                grid=False,
                title="% of respondents",
                format=".0%",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                titleY=5,
                titleX=7,
                titleFontWeight="normal",
                titleFontSize=10,
                titleAngle=0,
                titleAlign="left",
            ),
            scale=alt.Scale(domain=[0, 0.4]),
        ),
    )
)
text = bars.mark_text(dy=5, dx=1, align="center", baseline="top").encode(
    text=alt.Text("percent:Q", format=".0%"),
    color=alt.condition(
        datum.percent < 0.08,
        alt.ColorValue(None),
        alt.ColorValue("white"),
    ),
    y=alt.Y(
        "sum(percent):Q",
        stack="zero",
    ),
)

text2 = bars.mark_text(dy=-5, dx=1, align="center", baseline="bottom").encode(
    text=alt.Text("percent:Q", format=".0%"),
    color=alt.condition(
        datum.percent > 0.07,
        alt.ColorValue(None),
        alt.ColorValue(colors["eco-gray"]),
    ),
    y=alt.Y(
        "sum(percent):Q",
        stack="zero",
    ),
)
layer1 = (
    ((bars + text + text2).properties(width=alt.Step(40), height=250))
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1

alt.LayerChart(...)

# Fig 7

In [629]:
df = (
    pd.read_excel(
        "raw/Economic Observatory Graphs - Mar & Apr.xlsx",
        sheet_name="Commuting graphs",
        skiprows=30,
        nrows=7,
    )
    .dropna(how="all", axis=1)
    .dropna(how="all", axis=0)
)
df.columns = ["response", "percent"]

In [630]:
f = "fig7_transport_commuting"
f7 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f7.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f7 = df
df.head()

,response,percent
0,Car /\nMinibus /\nVan,0.63
1,Bus,0.15
2,Train,0.11
3,Bicycle,0.09
4,Tram / Tube,0.08


In [631]:
bars = (
    alt.Chart(f7)
    .mark_bar(height=20, opacity=0.9)
    .encode(
        color=alt.Color(
            "response:N",
            legend=None,
            scale=alt.Scale(
                domain=df["response"].unique(),
                range=category_color,
            ),
        ),
        y=alt.Y(
            "response:N",
            sort=[],
            axis=alt.Axis(
                grid=False,
                title="",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
            ),
        ),
        x=alt.X(
            "sum(percent):Q",
            sort=[],
            axis=alt.Axis(
                grid=False,
                title="% of respondents",
                format=".0%",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                titleAnchor="end",
                titleY=-17,
                titleFontWeight="normal",
                titleFontSize=10,
            ),
            scale=alt.Scale(domain=[0, 0.6]),
        ),
        order="response:N",
    )
)
text = bars.mark_text(dx=-5, dy=1, align="right").encode(
    text=alt.Text("percent:Q", format=".0%"),
    color=alt.condition(
        datum.percent < 0.08,
        alt.ColorValue(None),
        alt.ColorValue("white"),
    ),
    x=alt.X(
        "sum(percent):Q",
        stack="zero",
    ),
)

text2 = bars.mark_text(dx=5, dy=1, align="left").encode(
    text=alt.Text("percent:Q", format=".0%"),
    color=alt.condition(
        datum.percent > 0.07,
        alt.ColorValue(None),
        alt.ColorValue(colors["eco-gray"]),
    ),
    x=alt.X(
        "sum(percent):Q",
        stack="zero",
    ),
)
layer1 = (
    ((bars + text + text2).properties(height=alt.Step(25), width=300))
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1

alt.LayerChart(...)

# Fig 8

In [632]:
df = (
    pd.read_excel(
        "raw/Economic Observatory Graphs - Mar & Apr.xlsx",
        sheet_name="Commuting graphs",
        skiprows=49,
        nrows=6,
    )
    .dropna(how="all", axis=1)
    .dropna(how="all", axis=0)
)
df.columns = ["response", "percent"]

In [633]:
f = "fig8_future_days_commuting"
f8 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f8.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f8 = df
df.head()

,response,percent
0,0 days,0.248
1,1 day,0.099
2,2 days,0.206
3,3 days,0.197
4,4 days,0.105


In [634]:
bars = (
    alt.Chart(f8)
    .mark_bar(height=20, opacity=0.9)
    .encode(
        color=alt.Color(
            "response:N",
            legend=None,
            scale=alt.Scale(
                domain=df["response"].unique(),
                range=mhue_color + [colors["eco-dark-blue"]],
            ),
        ),
        y=alt.Y(
            "response:N",
            sort=[],
            axis=alt.Axis(
                grid=False,
                title="",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
            ),
        ),
        x=alt.X(
            "sum(percent):Q",
            sort=[],
            axis=alt.Axis(
                grid=False,
                title="% of respondents",
                format=".0%",
                labelColor=colors["eco-gray"],
                titleColor=colors["eco-gray"],
                tickColor=colors["eco-gray"],
                domainColor=colors["eco-gray"],
                titleAnchor="end",
                titleY=-17,
                titleFontWeight="normal",
                titleFontSize=10,
            ),
            scale=alt.Scale(domain=[0, 0.25]),
        ),
        order="response:N",
    )
)
text = bars.mark_text(dx=-5, dy=1, align="right").encode(
    text=alt.Text("percent:Q", format=".0%"),
    color=alt.condition(
        datum.percent < 0.08,
        alt.ColorValue(None),
        alt.ColorValue("white"),
    ),
    x=alt.X(
        "sum(percent):Q",
        stack="zero",
    ),
)

text2 = bars.mark_text(dx=5, dy=1, align="left").encode(
    text=alt.Text("percent:Q", format=".0%"),
    color=alt.condition(
        datum.percent > 0.08,
        alt.ColorValue(None),
        alt.ColorValue(colors["eco-gray"]),
    ),
    x=alt.X(
        "sum(percent):Q",
        stack="zero",
    ),
)
layer1 = (
    ((bars + text + text2).properties(height=alt.Step(25), width=300))
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1

alt.LayerChart(...)